# 05 — Rep Counting Baseline

Rule-based push-up counting using the elbow-angle state machine. Evaluate on labeled videos.


In [ ]:
# Setup
import sys
sys.path.insert(0, "..")


## TODO

Implement notebook content as we reach this phase.
